In [1]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from functools import lru_cache

In [2]:
base_url = "https://demo.hafas.de/openapi/vbb-proxy/"

Read secret access ID for VBB API from a file

In [3]:
ACCESS_ID = !cat ACCESS_ID.txt
ACCESS_ID = ACCESS_ID.s

In [5]:
@lru_cache(maxsize=500)
def req(endpoint):
    response = requests.get(endpoint)
    js = json.loads(response.text)
    return js

In [6]:
origin = (52.5159984,13.4575739)  # Frankfurter Allee 15
dest = (52.5219216,13.411026)  # Alexanderplatz

In [69]:
def append_access_id(endpoint):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "accessId={ACCESS_ID}".format(ACCESS_ID=ACCESS_ID)

def force_json_format(endpoint):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "format=json"

def set_latlon(endpoint, point, mode):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + \
            "{mode}CoordLat={point[0]}".format(mode=mode, point=point) + '&' + \
            "{mode}CoordLon={point[1]}".format(mode=mode, point=point)

def set_origin_latlon(endpoint, point):
    return set_latlon(endpoint, point, 'origin')

def set_dest_latlon(endpoint, point):
    return set_latlon(endpoint, point, 'dest')

def set_extid(endpoint, extid, mode):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "{}ExtId={}".format(mode,extid)

def set_origin_extid(endpoint, extid):
    return set_extid(endpoint, extid, 'origin')

def set_dest_extid(endpoint, extid):
    return set_extid(endpoint, extid, 'dest')

def append_date_and_time(endpoint, datetime_):
    date_ = datetime_.strftime('%Y-%m-%d')
    time_ = datetime_.strftime('%H:%M:%S')
    duration = 20  # mins
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "date={}&time={}&duration={}".format(date_, time_, duration)

def enable_walk_routes(endpoint):
    return enable_alt_routes(endpoint, 'Walk')

def enable_bike_routes(endpoint):
    return enable_alt_routes(endpoint, 'Bike')

def enable_alt_routes(endpoint, mode):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "total{0}=1,0,5000&origin{0}=1,0,5000&dest{0}=1,0,5000".format(mode)

Resolve origin and dest HAFAS locations using nearbystops

In [86]:
def get_closest_stop(point):
    endpoint = base_url + 'location.nearbystops'
    endpoint = append_access_id(endpoint)
    endpoint = force_json_format(endpoint)
    endpoint += '&originCoordLong={}&originCoordLat={}&maxNo=1&r=2000'.format(origin[1],origin[0])
    response = req(endpoint)
    for key, result in response.items():
        for location in result:
            # print(location['StopLocation'], location['StopLocation']['extId'])
            return location['StopLocation']
        
    return None

In [9]:
origin_ext = get_closest_stop(origin)
origin_extid = origin_ext['extId']
dest_extid = '900100003'

In [10]:
pt = (52.52448133333333, 13.46405325)
endpoint = base_url + 'location.nearbystops'
endpoint = append_access_id(endpoint)
endpoint = force_json_format(endpoint)
endpoint += '&originCoordLong={}&originCoordLat={}&maxNo=1&r=1000'.format(pt[1],pt[0])
response = req(endpoint)
print(response)

{'stopLocationOrCoordLocation': [{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'name': 'S8', 'line': 'S8', 'lineId': 'S8', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-8', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S41', 'line': 'S41', 'lineId': 'S41', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-3', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S42', 'line': 'S42', 'lineId': 'S42', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-3', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S85', 'line': 'S85', 'lineId': 'S85', 'catOut': 'S', 'cls': '1', 'catOutS': 'S12', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': '156', 'line': '156', 'lineId': '156', 'catOut': 'Bus', 'cls': '8', 'catOutS': 'B', 'catOutL': 'Bus'}, {'icon': {'res': 'prod_gen'}, 'name': '240', 'line': '240', 'lineId': '240', 'catOut': 'Bus', 'cls': '8', 'catOutS': 'B', 'catOutL': 'Bus'}], 'id': 'A=1@O=S Storkower Str. (Berlin)@X=13464834@Y=52524052@u=0@U=86@L=900110012@', 'extId': '

Request all trips between origin and dest

In [81]:
def total_travel_time(response):
    ttt = []
    for trip in response['Trip']:
        leglist = trip['LegList']

        for key in leglist:
            travel_time = timedelta()
            for leg in leglist['Leg']:
                # print(leg, len(leg))
                leg_origin, leg_dest = leg['Origin'], leg['Destination']
                # print(leg_origin['time'], leg_dest['time'])
                leg_origin_time = datetime.strptime(leg_origin['time'], '%H:%M:%S')
                leg_dest_time = datetime.strptime(leg_dest['time'], '%H:%M:%S')
                print(leg_dest)
                td = timedelta() + leg_dest_time - leg_origin_time
                travel_time += td
            # print('TOTAL TRAVEL TIME: {}'.format(travel_time))
            ttt.append(travel_time)
    return ttt

def all_trips(response):
    trips = []
    try:
        for trip in response['Trip']:
            leglist = trip['LegList']

            for key in leglist:
                travel_time = timedelta()
                legs = []
                for leg in leglist['Leg']:
                    # print(leg, len(leg))
                    leg_origin, leg_dest = leg['Origin'], leg['Destination']
                    if leg_origin['name'] not in legs:
                        legs.append(leg_origin['name'])
                    # print(leg_origin['time'], leg_dest['time'])
                    leg_origin_time = datetime.strptime(leg_origin['time'], '%H:%M:%S')
                    leg_dest_time = datetime.strptime(leg_dest['time'], '%H:%M:%S')
                    td = timedelta() + leg_dest_time - leg_origin_time
                    travel_time += td
                    legs.append(leg_dest['name'])
                # print('TOTAL TRAVEL TIME: {}'.format(travel_time))
                trips.append((travel_time, legs))
    except KeyError as e:
        if response['errorCode'] == 'SVC_NO_RESULT':
            # print('Could not find a trip')
            pass
        elif response['errorCode'] == 'SVC_LOC_EQUAL': # start and dest are equal -> 0 time
            trips.append((timedelta(seconds=0.0), ()),)
        else:
            print(e)
        
    return trips

def compute_travel_times(origin_extid, dest_extid, datetime_=None):
    endpoint = base_url + 'trip'
    endpoint = append_access_id(endpoint)
    endpoint = force_json_format(endpoint)
    endpoint = set_origin_extid(endpoint, origin_extid)
    endpoint = set_dest_extid(endpoint, dest_extid)
    endpoint = enable_walk_routes(endpoint)
    if datetime_ is not None:
        endpoint = append_date_and_time(endpoint, datetime_)
    return total_travel_time(req(endpoint))

def compute_trips(origin_extid, dest_extid, datetime_=None):
    endpoint = base_url + 'trip'
    endpoint = append_access_id(endpoint)
    endpoint = force_json_format(endpoint)
    endpoint = set_origin_extid(endpoint, origin_extid)
    endpoint = set_dest_extid(endpoint, dest_extid)
    endpoint = enable_walk_routes(endpoint)
    endpoint = enable_bike_routes(endpoint)
    if datetime_ is not None:
        endpoint = append_date_and_time(endpoint, datetime_)
    return all_trips(req(endpoint))

In [29]:
compute_trips(origin_extid, dest_extid, datetime_=datetime(2020,4,22,8,0))

[(datetime.timedelta(seconds=600),
  ['Wolliner Str. (Berlin)',
   'U Eberswalder Str. (Berlin)',
   'S+U Alexanderplatz (Berlin) [U2]']),
 (datetime.timedelta(seconds=420),
  ['Wolliner Str. (Berlin)',
   'U Bernauer Str. (Berlin)',
   'S+U Alexanderplatz (Berlin) [U8]']),
 (datetime.timedelta(seconds=600),
  ['Wolliner Str. (Berlin)',
   'U Eberswalder Str. (Berlin)',
   'S+U Alexanderplatz (Berlin) [U2]']),
 (datetime.timedelta(seconds=420),
  ['Wolliner Str. (Berlin)',
   'U Bernauer Str. (Berlin)',
   'S+U Alexanderplatz (Berlin) [U8]']),
 (datetime.timedelta(seconds=600),
  ['Wolliner Str. (Berlin)',
   'U Eberswalder Str. (Berlin)',
   'S+U Alexanderplatz (Berlin) [U2]'])]

In [13]:
compute_travel_times(origin_extid, dest_extid)

{'Notes': {'Note': [{'value': 'Verantwortlicher DINO-Haltestellenbereich', 'key': 'IS', 'type': 'A', 'priority': 999, 'txtN': 'Verantwortlicher DINO-Haltestellenbereich'}]}, 'name': 'S+U Alexanderplatz (Berlin) [U5]', 'type': 'ST', 'id': 'A=1@O=S+U Alexanderplatz (Berlin) [U5]@X=13413110@Y=52521607@U=86@L=900100704@', 'extId': '900100704', 'lon': 13.41311, 'lat': 52.521607, 'routeIdx': 19, 'prognosisType': 'PROGNOSED', 'time': '00:49:00', 'date': '2020-04-25', 'rtTime': '00:49:00', 'rtDate': '2020-04-25'}
{'Notes': {'Note': [{'value': 'Verantwortlicher DINO-Haltestellenbereich', 'key': 'IS', 'type': 'A', 'priority': 999, 'txtN': 'Verantwortlicher DINO-Haltestellenbereich'}]}, 'name': 'S+U Alexanderplatz (Berlin) [U5]', 'type': 'ST', 'id': 'A=1@O=S+U Alexanderplatz (Berlin) [U5]@X=13413110@Y=52521607@U=86@L=900100704@', 'extId': '900100704', 'lon': 13.41311, 'lat': 52.521607, 'routeIdx': 19, 'prognosisType': 'PROGNOSED', 'time': '00:59:00', 'date': '2020-04-25', 'rtTime': '00:59:00', 'r

[datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=300),
 datetime.timedelta(seconds=300),
 datetime.timedelta(seconds=300)]

Consider distance from the requested GPS coordinate to the nearest stop

In [14]:
origin = (52.517602, 13.458108)

In [15]:
origin_closest_stop = get_closest_stop(origin)
origin_extid = origin_closest_stop['extId']
origin_closest_stop_coord = (origin_closest_stop['lat'], origin_closest_stop['lon'])

In [16]:
import math

def distance_utf(point1, point2):
    def toRadians(deg):
        return deg * math.pi / 180.0
    
    R = 6371e3; # metres
    lat1, lon1 = point1
    lat2, lon2 = point2
    phi1 = toRadians(lat1)
    phi2 = toRadians(lat2)
    delta_phi = toRadians(lat2-lat1)
    delta_lambda = toRadians(lon2-lon1)

    a = math.sin(delta_phi/2) * math.sin(delta_phi/2) + \
            math.cos(phi1) * math.cos(phi2) * \
            math.sin(delta_lambda/2) * math.sin(delta_lambda/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    d = R * c
    return d

In [17]:
travel_times = compute_travel_times(origin_extid, dest_extid)
travel_times

{'Notes': {'Note': [{'value': 'Aufzug', 'key': 'AT', 'type': 'A', 'priority': 300, 'txtN': 'Aufzug'}, {'value': 'Blindenleitstreifen', 'key': 'BL', 'type': 'A', 'priority': 300, 'txtN': 'Blindenleitstreifen'}, {'value': 'Verantwortlicher DINO-Haltestellenbereich', 'key': 'IS', 'type': 'A', 'priority': 999, 'txtN': 'Verantwortlicher DINO-Haltestellenbereich'}]}, 'name': 'U Frankfurter Tor (Berlin)', 'type': 'ST', 'id': 'A=1@O=U Frankfurter Tor (Berlin)@X=13454083@Y=52515773@U=86@L=900120008@', 'extId': '900120008', 'lon': 13.454083, 'lat': 52.515773, 'routeIdx': 20, 'prognosisType': 'PROGNOSED', 'time': '00:47:00', 'date': '2020-04-25', 'rtTime': '00:47:00', 'rtDate': '2020-04-25'}
{'Notes': {'Note': [{'value': 'Verantwortlicher DINO-Haltestellenbereich', 'key': 'IS', 'type': 'A', 'priority': 999, 'txtN': 'Verantwortlicher DINO-Haltestellenbereich'}]}, 'name': 'S+U Alexanderplatz (Berlin) [U5]', 'type': 'ST', 'id': 'A=1@O=S+U Alexanderplatz (Berlin) [U5]@X=13413110@Y=52521607@U=86@L=900

[datetime.timedelta(seconds=480),
 datetime.timedelta(seconds=420),
 datetime.timedelta(seconds=420),
 datetime.timedelta(seconds=420),
 datetime.timedelta(seconds=420)]

In [18]:
distance_utf(origin, origin_closest_stop_coord)

284.97535065497823

In [19]:
WALK_VELOCITY = 3000/3600.0 # m/s
distance_to_closest_stop = distance_utf(origin, origin_closest_stop_coord)
travel_times_adjusted = []
for tt in travel_times:
    travel_times_adjusted.append(tt + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY))
travel_times_adjusted

[datetime.timedelta(seconds=821, microseconds=970421),
 datetime.timedelta(seconds=761, microseconds=970421),
 datetime.timedelta(seconds=761, microseconds=970421),
 datetime.timedelta(seconds=761, microseconds=970421),
 datetime.timedelta(seconds=761, microseconds=970421)]

Record data into a CSV file for later rendering

In [20]:
df = pd.DataFrame(columns=['latitude', 'longitude', 'min travel time'])

origins = [
    (52.5159984,13.4575739),
    (52.517602, 13.458108)
]
WALK_VELOCITY = 3000/3600.0 # m/s

for origin in origins:
    origin_closest_stop = get_closest_stop(origin)
    distance_to_closest_stop = distance_utf(origin, origin_closest_stop_coord)
    origin_extid = origin_closest_stop['extId']
    origin_closest_stop_point = (origin_closest_stop['lat'], origin_closest_stop['lon'])
    
    dt = datetime(2020, 5, 1, 5, 0, 0)
    trips = compute_trips(origin_extid, dest_extid, dt)
    
    trips_adjusted = []
    for travel_time, trip in trips:
        trips_adjusted.append((travel_time + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY), trip))

    shortest_trip = sorted(trips_adjusted, key=lambda x:x[0])[0]
    df = df.append({'latitude': origin[0], 'longitude': origin[1], 'min travel time': shortest_trip[0].total_seconds(), 'shortest trip': ' > '.join(shortest_trip[1])}, ignore_index=True)

In [21]:
df

,latitude,longitude,min travel time,shortest trip
0,52.515998,13.457574,773.557670,U Frankfurter Tor (Berlin) > S+U Alexanderplat...
1,52.517602,13.458108,761.970421,Bersarinplatz (Berlin) > U Frankfurter Tor (Be...


In [22]:
df.to_csv('hafas_vbb.csv')

# Compute travel time for an equally spaced rectangular area

In [82]:
topleft = (52.54, 13.40)
bottomright = (52.50, 13.50)
num_points = 15
WALK_VELOCITY = 3.0/3.6

df = pd.DataFrame(columns=['latitude', 'longitude', 'min travel time'])

origins = []
lat_arr = np.linspace(bottomright[0], topleft[0], num_points)
lon_arr = np.linspace(bottomright[1], topleft[1], num_points)
for lat in lat_arr:
    for lon in lon_arr:
        origins.append((lat, lon))

for origin in origins:
    origin_closest_stop = get_closest_stop(origin)
    origin_extid = origin_closest_stop['extId']
    origin_closest_stop_point = (origin_closest_stop['lat'], origin_closest_stop['lon'])
    distance_to_closest_stop = distance_utf(origin, origin_closest_stop_point)
    
    dt = datetime(2020, 5, 1, 5, 0, 0)
    trips = compute_trips(origin_extid, dest_extid, dt)
    trips_adjusted = []
    for travel_time, trip in trips:
        trips_adjusted.append((travel_time + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY), trip))

    sorted_trips = sorted(trips_adjusted, key=lambda x:x[0])
    if sorted_trips:
        shortest_trip = sorted_trips[0]
        shortest_trip_time = shortest_trip[0].total_seconds()
    else:
        shortest_trip = (distance_utf(origin, dest)/WALK_VELOCITY, ['No trip found'])
        shortest_trip_time = distance_utf(origin, dest)/WALK_VELOCITY
    shortest_trip_itinerary = shortest_trip[1]
    
    df = df.append({'latitude': origin[0], 'longitude': origin[1], 'min travel time': shortest_trip_time, 'shortest trip': ' > '.join(shortest_trip_itinerary)}, ignore_index=True)
    print('.', end='')

.................................................................................................................................................................................................................................

In [83]:
df

,latitude,longitude,min travel time,shortest trip
0,52.50,13.500000,1073.610024,Michiganseestr. (Berlin) > S Ostkreuz Bhf (Ber...
1,52.50,13.492857,832.495337,Lückstr./Weitlingstr. (Berlin) > S Ostkreuz Bh...
2,52.50,13.485714,1446.585478,S Nöldnerplatz/Schlichtallee (Berlin) > S Nöld...
3,52.50,13.478571,821.904976,S Rummelsburg (Berlin) > S+U Warschauer Str. (...
4,52.50,13.471429,1198.938142,S Rummelsburg/Hauptstr. EV (Berlin) > S Rummel...
...,...,...,...,...
220,52.54,13.428571,738.537550,Winsstr. (Berlin) > Prenzlauer Allee/Danziger ...
221,52.54,13.421429,640.595207,Husemannstr. (Berlin) > U Eberswalder Str. (Be...
222,52.54,13.414286,520.258308,U Eberswalder Str. (Berlin) > S+U Alexanderpla...
223,52.54,13.407143,579.211847,Friedrich-Ludwig-Jahn-Sportpark (Berlin) > U E...


In [85]:
df.to_csv('hafas_vbb.csv')